In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
%matplotlib notebook
from tqdm import tqdm #プログレスバー
import csv
from MOEAD import *
import random

In [2]:
class Knapsack:
    def __init__(self, items_file_name, knapsack, item_num):
        #物品データ読み込み
        with open(items_file_name, 'r') as f:
            line = f.readline()
            print(line)
            self.k_items = []
            self.capacities = []
            for k in range(knapsack):
                line = f.readline() # "="
                if not("=" in line):
                    print("読み込みが正しく行えませんでした k:%d, i:%d" % (k, i))
                    exit()
                line = f.readline() # "knapsack n:"
                print(line)
                line = f.readline() # "capacity"
                self.capacities.append(float(line[11:]))
                items = []
                for i in range(item_num):
                    item = []
                    line = f.readline() # "item"
                    item.append(line[1:-1])
                    line = f.readline() # "weight"
                    item.append(int(line[10:]))
                    line = f.readline() # "profit"
                    item.append(int(line[10:]))
                    items.append(item)
                self.k_items.append(items)
                print(items)
                
        self.sum_profit = []
        for i in range(knapsack):
            self.sum_profit.append(sum([item[2] for item in self.k_items[i][:]]))

In [3]:
# メイン処理記述
items_file_name = "knapsack_500_2.txt"

H = 200 # 分割数パラメータ
T = 10 #近傍数
G = 1000 # 世代数
m = 2 #目的数
n = 500 #アイテム数
phi = 0.5 # Φ 実行可能率
p_m = 1.0 / n # 突然変異率

In [4]:
knapsack = Knapsack(items_file_name, m, n)

knapsack problem specification (2 knapsacks, 500 items)

knapsack 1:

[['item 1:', 68, 100], ['item 2:', 80, 53], ['item 3:', 93, 48], ['item 4:', 14, 38], ['item 5:', 89, 72], ['item 6:', 15, 70], ['item 7:', 37, 75], ['item 8:', 41, 11], ['item 9:', 76, 87], ['item 10:', 26, 70], ['item 11:', 25, 88], ['item 12:', 93, 32], ['item 13:', 17, 78], ['item 14:', 29, 72], ['item 15:', 53, 83], ['item 16:', 27, 18], ['item 17:', 71, 30], ['item 18:', 26, 12], ['item 19:', 99, 88], ['item 20:', 87, 69], ['item 21:', 12, 39], ['item 22:', 60, 68], ['item 23:', 98, 28], ['item 24:', 93, 33], ['item 25:', 81, 26], ['item 26:', 74, 89], ['item 27:', 17, 31], ['item 28:', 16, 42], ['item 29:', 75, 88], ['item 30:', 84, 87], ['item 31:', 18, 75], ['item 32:', 61, 65], ['item 33:', 49, 45], ['item 34:', 65, 33], ['item 35:', 55, 47], ['item 36:', 23, 40], ['item 37:', 96, 30], ['item 38:', 56, 75], ['item 39:', 30, 25], ['item 40:', 16, 14], ['item 41:', 74, 49], ['item 42:', 63, 66], ['item 43:', 

In [5]:
#突然変異
def mutation_func(x, mutation):
    for i in range(len(x)):
        if random.random() < mutation:
            x[i] = (x[i] + 1) % 2
    return x

# 評価関数
def func_1(x):
    weight = 0
    profit = 0
    for i, item in enumerate(knapsack.k_items[0]):
        weight = weight + item[1] * x[i]
        profit = profit + item[2] * x[i]
        
    # 修復法
    while knapsack.capacities[0] < weight:
        is_not_changed = True
        while is_not_changed:
            i = random.randrange(n)
            if x[i] == 1:
                x[i] == 0
                weight = weight - knapsack.k_items[0][i][1]
                profit = profit - knapsack.k_items[0][i][2]
                is_not_changed = False
                
    return knapsack.sum_profit[0] - profit
                
def func_2(x):
    weight = 0
    profit = 0
    for i, item in enumerate(knapsack.k_items[1]):
        weight = weight + item[1] * x[i]
        profit = profit + item[2] * x[i]
        
    # 修復法
    while knapsack.capacities[1] < weight:
        is_not_changed = True
        while is_not_changed:
            i = random.randrange(n)
            if x[i] == 1:
                x[i] == 0
                weight = weight - knapsack.k_items[1][i][1]
                profit = profit - knapsack.k_items[1][i][2]
                is_not_changed = False
                
    return knapsack.sum_profit[1] - profit
        

fs = [func_1, func_2]

# x初期化関数
def init_x_func(L):
    x = []
    for i in range(len(L)):
        x_i = []
        for j in range(n):
            # 0.7の確率で0
            x_i.append(0 if random.random() < 0.7 else 1)
        x.append(x_i)
    return x

In [6]:
cross_over = OnePointCrossOver(n, int(n/2))
mutation = Mutation(mutation_func ,p_m)
moead = MOEAD(m, H, T, fs, cross_over, mutation)
moead.set_init_x_func(init_x_func)

m:2, H:200, N:201, T:10


In [7]:
moead.init_phase()
x = moead.solution_search_phase(10)

[1 0 0 0 0 0 0 0 0 0 1 1 0 1 0 1 0 1 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1
 1 0 1 0 0 0 0 0 0 1 1 0 0 1 1 0 1 0 1 0 0 0 0 0 1 0 1 0 1 0 0 1 0 0 0 1 1
 1 0 1 1 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 1 1 1 1 0 0 0 0 0
 1 0 1 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 1 0 0 1 0 0 1 0 0 0 1 0 1 0 1 0 0 0
 0 0 0 1 0 0 0 1 0 1 0 0 0 0 0 0 0 1 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 1 1 0 1 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 1 0 0 1 1 1 1 0 0 0 0 0 1 1 0 0
 0 0 0 0 0 0 0 1 0 0 1 1 0 0 1 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 1 0 0 0 0 1 0
 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 1 0 0 1 0 0 0 1 1 0 0 0 1 1 0 1 0 0 0 0 0 0
 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 1 0 0 1 0 1 1 0 0 0 1 0
 0 0 1 0 0 0 1 0 0 0 0 0 1 0 1 0 1 0 0 0 0 0 0 1 0 0 0 1 1 1 0 0 1 0 0 0 1
 1 1 1 1 0 1 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 1 0 1 0 0 0 0 0 1 0 0
 0 1 0 0 1 0 1 0 0 1 0 0 0 1 0 1 1 1 0 1 0 0 0 0 0 0 1 1 1 1 0 1 1 0 0 1 0
 0 1 0 0 0 0 0 1 0 0 1 1 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0
 0 1 0 0 1 1 1 0 0 0 0 1 

 0 1 0 0 1 1 1 0 0 0 0 1 1 1 0 0 0 0 1]
[1 0 0 0 0 0 0 0 0 0 1 1 0 1 0 1 0 1 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1
 1 0 1 0 0 0 0 0 0 1 1 0 0 1 1 0 1 0 1 0 0 0 0 0 1 0 1 0 1 0 0 1 0 0 0 1 1
 1 0 1 1 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 1 1 1 1 0 0 0 0 0
 1 0 1 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 1 0 0 1 0 0 1 0 0 0 1 0 1 0 1 0 0 0
 0 0 0 1 0 0 0 1 0 1 0 0 0 0 0 0 0 1 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 1 1 0 1 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 1 0 0 1 1 1 1 0 0 0 0 0 1 1 0 0
 0 0 0 0 0 0 0 1 0 0 1 1 0 0 1 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 1 0 0 0 0 1 0
 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 1 0 0 1 0 0 0 1 1 0 0 0 1 1 0 1 0 0 0 0 0 0
 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 1 0 0 1 0 1 1 0 0 0 1 0
 0 0 1 0 0 0 1 0 0 0 0 0 1 0 1 0 1 0 0 0 0 0 0 1 0 0 0 1 1 1 0 0 1 0 0 0 1
 1 1 1 1 0 1 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 1 0 1 0 0 0 0 0 1 0 0
 0 1 0 0 1 0 1 0 0 1 0 0 0 1 0 1 1 1 0 1 0 0 0 0 0 0 1 1 1 1 0 1 1 0 0 1 0
 0 1 0 0 0 0 0 1 0 0 1 1 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0

[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 

NameError: name 'k' is not defined